In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys

In [3]:
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [6]:
import IPython.display as ipd
from IPython.display import Audio
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization , GRU
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

In [8]:
pip show keras

Name: keras
Version: 2.12.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: c:\users\user\appdata\roaming\python\python39\site-packages
Requires: 
Required-by: tensorflow-intel
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install --upgrade keras

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 1.0/1.0 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 3.0.5 which is incompatible.


In [12]:
from keras.utils import pad_sequences

In [13]:
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import tensorflow as tf 
print ("Done")

Done


In [14]:
!apt-get update
!apt-get install -y libsndfile1

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
ravdess = "./RAVDESS/audio_speech_actors_01-24/"
ravdess_directory_list = os.listdir(ravdess)
print(ravdess_directory_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [18]:
file_emotion = []
file_path = []
for i in ravdess_directory_list:
    actor = os.listdir(ravdess + i)
    for f in actor:
        part = f.split('.')[0].split('-')
        file_emotion.append(int(part[2]))
        file_path.append(ravdess + i + '/' + f)

In [19]:
print(actor[0])
print(part[0])
print(file_path[0])
print(int(part[2]))
print(f)

03-01-01-01-01-01-24.wav
03
./RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav
8
03-01-08-02-02-02-24.wav


In [20]:
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)
ravdess_df.Emotions.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust',
                             8:'surprise'},
                            inplace=True)
print(ravdess_df.head())
print("______________________________________________")
print(ravdess_df.tail())
print("_______________________________________________")
print(ravdess_df.Emotions.value_counts())

  Emotions                                               Path
0  neutral  ./RAVDESS/audio_speech_actors_01-24/Actor_01/0...
1  neutral  ./RAVDESS/audio_speech_actors_01-24/Actor_01/0...
2  neutral  ./RAVDESS/audio_speech_actors_01-24/Actor_01/0...
3  neutral  ./RAVDESS/audio_speech_actors_01-24/Actor_01/0...
4  neutral  ./RAVDESS/audio_speech_actors_01-24/Actor_01/0...
______________________________________________
      Emotions                                               Path
1435  surprise  ./RAVDESS/audio_speech_actors_01-24/Actor_24/0...
1436  surprise  ./RAVDESS/audio_speech_actors_01-24/Actor_24/0...
1437  surprise  ./RAVDESS/audio_speech_actors_01-24/Actor_24/0...
1438  surprise  ./RAVDESS/audio_speech_actors_01-24/Actor_24/0...
1439  surprise  ./RAVDESS/audio_speech_actors_01-24/Actor_24/0...
_______________________________________________
neutral     288
happy       192
sad         192
angry       192
fear        192
disgust     192
surprise    192
Name: Emotions, dtype: in

In [38]:
def extract_features(RAVDESS):
    max_pad_len = 174
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs

In [27]:
from sklearn.preprocessing import LabelEncoder

In [29]:
from keras.utils import np_utils

In [39]:
X = np.array(ravdess_df.features.tolist())
y = np.array(ravdess_df.Emotions.tolist())
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
from keras.layers import TimeDistributed

In [34]:
from keras.regularizers import l2

In [40]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(40, 174)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(TimeDistributed(Dense(32, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(TimeDistributed(Dense(16, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(TimeDistributed(Dense(8, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])